Jupyter Notebook dedicated to the preprocessing of a dataset loaded from `output_file.csv`. The initial steps involve importing necessary libraries, such as pandas for data manipulation and re for regular expressions, and loading the dataset into a pandas DataFrame.

In [ ]:
import pandas as pd
import re

df = pd.read_csv('data/output_file.csv')

Preview the `.csv` file to see if its corresponding.

In [ ]:
df.head(10)

Cleans and standardizing of the data for further analysis. In this section, we focus on the `experience` column, converting it to a string format to handle any NaN values seamlessly. 
A function is defined to extract numerical values from this column, aiming to capture the YoE in a uniform format.

In [ ]:
df['experience'] = df['experience'].astype(str)  # Convert to string to handle NaN values

# Define a function to extract the numerical value from the experience column
def extract_experience(text):
    match = re.search(r'\+?\s*(\d+)', str(text))
    if match:
        return match.group(1)
    else:
        return None

# Apply the function to the 'Expérience' column
df['experience'] = df['experience'].apply(extract_experience)

df.head(5)

Standardize the `salaire` (salary) column by first converting all entries to strings to facilitate the extraction of salary ranges using regular expressions. 
The `extract_average_salary` function identifies and calculates the average of salary ranges, adjusting monthly salaries to their yearly equivalents for consistency. 
Finally, the processed salaries are rounded to two decimal places, and the first 20 rows of the DataFrame are displayed to verify the modifications.

In [ ]:
df['salaire'] = df['salaire'].astype(str)

def extract_average_salary(text):
  text = text.replace(' ','')
  matches = re.findall(r'(\d[\d ,.]*)\s*-\s*(\d[\d ,.]*)\s*(?:EUR)?\s*(?:par)?\s*(an|mois)?', str(text), flags=re.IGNORECASE)
  if matches:
    start_salary, end_salary, unit = matches[0]
    start_salary, end_salary = map(float, [start_salary.replace(',','.'), end_salary.replace(',','.')])

    if unit and unit.lower() == 'mois':
        return (start_salary + end_salary) / 2 * 12  # Convert monthly to yearly
    else:
        return (start_salary + end_salary) / 2
  else:
    return None

df['salaire'] = df['salaire'].apply(extract_average_salary)
df['salaire'] = df['salaire'].round(2)
df.head(20)

Compute the mean salary and fill the `NaN`values with it to allow for further analysis without calculation errors.

In [ ]:
average_salaire = df['salaire'].mean()

# Replace NaN values in the 'salaire' column with the average value
df['salaire'].fillna(average_salaire, inplace=True)

Round the salary value to the nearest round number, and filter the dataframe to only include salary values greater than or equal to 1000.

In [ ]:
df['salaire'] = df['salaire'].round(0)
df = df[df['salaire'] >= 1000]

This code standardizes location names for Paris and Lyon in the `location` column by removing district information. It replaces specific district references with a generalized city name for Paris and Lyon to avoid redundance.

In [ ]:
df['location'].replace('Paris 1er - 75', 'Paris - 75', inplace=True)
df['location'].replace('Lyon 1er - 69', 'Lyon - 69', inplace=True)
df[df['location']=='Lyon - 69'].count()

Split the `location` column into two new columns, `city` and `department`, based on a dash delimiter, and then removes the original `location` column from the DataFrame to only keep the `department` number.

In [ ]:
df[['city', 'department']] = df['location'].str.split(r' - ', expand=True)

# Drop the original 'location' column
df.drop(columns=['location'], inplace=True)


Identify rows where the `department` column is null. After dropping rows with null `department` values, verify the removal by checking for nulls again.

In [ ]:
df[df['department'].isnull()]
df.dropna(subset=['department'], inplace=True)
df[df['department'].isnull()]

Convert string representations of lists in the `skills` column back into actual list objects using `ast.literal_eval`, replacing NaN values with empty lists.


In [ ]:
import ast

df['skills'] = df['skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])

Define a function to remove duplicate entries from lists in the `skills` column. Apply it, and convert all skills to lowercase for consistency matching.


In [ ]:
def remove_duplicates(lst):
    return list(set(lst))

# Apply the function to remove duplicates
df['skills'] = df['skills'].apply(remove_duplicates)

df['skills'] = df['skills'].apply(lambda x: [skill.lower() for skill in x])

Filter the DataFrame to include only rows where the list in the `skills` column contains more than 20 items. Print the shape of the DataFrame to see if the operation worked correctly.

In [ ]:
filtered_rows = df[df['skills'].apply(len) > 20]
filtered_rows.shape

Download the processed DataFrame in the `finalData.csv` csv file.

In [ ]:
df.to_csv('finalData.csv', index=False)